In [ ]:
import requests
import io
import pandas as pd 

In [19]:
df = pd.read_csv(r"C:\Users\Philippa\Documents\GitHub\crystalline-mining\scraped_papers.csv")

df.head()

,arxiv id,title,published,author(s),abs link,pdf link,journal ref,comment,abstract
0,1606.07779v1,Natural calcium carbonate for biomedical appli...,2016-06-23T13:01:43Z,Sonali S. Sali,http://arxiv.org/abs/1606.07779v1,http://arxiv.org/pdf/1606.07779v1,No journal ref found,M.Tech (Biotechnology) Dissertation Thesis pro...,Sea shells are found to be a very rich natural...
1,2208.06361v1,Hyperbolic Molecular Representation Learning f...,2022-07-06T20:20:29Z,"Ke Yu, Shyam Visweswaran, Kayhan Batmanghelich",http://arxiv.org/abs/2208.06361v1,http://arxiv.org/pdf/2208.06361v1,No journal ref found,Accepted by NeurIPS workshop 2020. arXiv admin...,Learning accurate drug representations is esse...
2,2409.11712v1,Revolutionizing Pharmaceutical Manufacturing: ...,2024-09-18T05:35:58Z,"Rahul Kumar, Vikram Singh, Priya Gupta",http://arxiv.org/abs/2409.11712v1,http://arxiv.org/pdf/2409.11712v1,No journal ref found,submitted to IMECE 2024,The advent of 3D printing has transformed the ...
3,2006.00986v1,Semi-Supervised Hierarchical Drug Embedding in...,2020-06-01T14:46:31Z,"Ke Yu, Shyam Visweswaran, Kayhan Batmanghelich",http://arxiv.org/abs/2006.00986v1,http://arxiv.org/pdf/2006.00986v1,No journal ref found,No comment found,Learning accurate drug representation is essen...
4,1801.07691v2,Drug Selection via Joint Push and Learning to ...,2018-01-23T18:26:54Z,"Yicheng He, Junfeng Liu, Xia Ning",http://arxiv.org/abs/1801.07691v2,http://arxiv.org/pdf/1801.07691v2,IEEE/ACM Transactions on Computational Biology...,No comment found,Selecting the right drugs for the right patien...


In [20]:
# grab the urls from the 'pdf link' column and store them in a list
pdf_url_list = df["pdf link"].tolist()
print(pdf_url_list)

['http://arxiv.org/pdf/1606.07779v1', 'http://arxiv.org/pdf/2208.06361v1', 'http://arxiv.org/pdf/2409.11712v1', 'http://arxiv.org/pdf/2006.00986v1', 'http://arxiv.org/pdf/1801.07691v2', 'http://arxiv.org/pdf/2007.07886v2', 'http://arxiv.org/pdf/1912.03702v1', 'http://arxiv.org/pdf/2404.03516v1', 'http://arxiv.org/pdf/2205.11285v1', 'http://arxiv.org/pdf/2210.00802v3', 'http://arxiv.org/pdf/1711.04402v1', 'http://arxiv.org/pdf/1805.05593v1', 'http://arxiv.org/pdf/2012.11599v1', 'http://arxiv.org/pdf/2308.13891v1', 'http://arxiv.org/pdf/2006.14002v1', 'http://arxiv.org/pdf/1511.04431v1', 'http://arxiv.org/pdf/1904.00313v1', 'http://arxiv.org/pdf/2405.14545v1', 'http://arxiv.org/pdf/2506.01478v1', 'http://arxiv.org/pdf/2110.11523v1', 'http://arxiv.org/pdf/2005.10644v1', 'http://arxiv.org/pdf/2307.01507v2', 'http://arxiv.org/pdf/2107.06773v2', 'http://arxiv.org/pdf/1905.06237v1', 'http://arxiv.org/pdf/1911.06356v2', 'http://arxiv.org/pdf/1908.10563v1', 'http://arxiv.org/pdf/2411.01422v1', 

In [21]:
from pathlib import Path
import requests

data_path = Path("data/")
pdf_path = data_path / "paper_pdfs"

if pdf_path.is_dir():
    print(f"{pdf_path} already exists.")
else:
    print(f"Creating {pdf_path}...")
    pdf_path.mkdir(parents=True, exist_ok=True)
    
i = 0

for url in pdf_url_list:
    if "pdf" in url:
        i += 1
        print(f"Downloading file {i}")
        request = requests.get(url)
        with open(pdf_path / f"research_paper_{i}.pdf", "wb") as f:
            f.write(request.content)
        print("File downloaded successfully.")

Creating data\paper_pdfs...
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloaded successfully.
File downloa

In [22]:
import pdfplumber

def convert_pdf_to_text_and_metadata(path):
    with pdfplumber.open(path, strict_metadata=True) as pdf:
        meta = pdf.metadata
        
        paged_pdf = pdf.pages
        text = [] # pdf has become a list containing one pdfplumber.Page instance per page. Need to loop through each page in the list and append text to a new list
        for i in range(len(paged_pdf)):
            words = paged_pdf[i].extract_text(x_tolerance=3, 
                                       x_tolerance_ratio=None, 
                                       y_tolerance=3, 
                                       layout=False, 
                                       x_density=7.25, 
                                       y_density=13, 
                                       line_dir_render=None, 
                                       char_dir_render=None)
            text.append(words)
        return text, meta

In [74]:
import pandas as pd
from pathlib import Path

pdf_files_dir = r"C:\Users\Philippa\Documents\GitHub\crystalline-mining\data"

parsed_paper_info = {"pdf_text":[],
                     "metadata":[]}

pathlist = Path(pdf_files_dir).glob("**/*.pdf")
for path in pathlist:
    path_in_str = str(path)
    print(path_in_str)
    text, metadata = convert_pdf_to_text_and_metadata(path_in_str)
    parsed_paper_info["pdf_text"].append(text)
    parsed_paper_info["metadata"].append(metadata)
    
df_parsed = pd.DataFrame.from_dict(parsed_paper_info)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

C:\Users\Philippa\Documents\GitHub\crystalline-mining\data\paper_pdfs\research_paper_1.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

C:\Users\Philippa\Documents\GitHub\crystalline-mining\data\paper_pdfs\research_paper_10.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


...

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

C:\Users\Philippa\Documents\GitHub\crystalline-mining\data\paper_pdfs\research_paper_98.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

C:\Users\Philippa\Documents\GitHub\crystalline-mining\data\paper_pdfs\research_paper_99.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [82]:
df_parsed.head()

,pdf_text,metadata
0,[NATURAL CALCIUM CARBONATE FOR BIOMEDICAL\nAPP...,"{'Author': 'ismail - [2010]', 'Creator': 'Micr..."
1,[ProceedingsofMachineLearningResearchLEAVEUNSE...,"{'Author': '', 'CreationDate': 'D:202404290022..."
2,[Preprint\nAdvances in Chemical Physics\nVol. ...,"{'CreationDate': 'D:20060424095709Z', 'ModDate..."
3,[Enhancing Drug-Drug Interaction Extraction fr...,"{'Author': '', 'CreationDate': 'D:201805160029..."
4,[BERTChem-DDI : Improved Drug-Drug Interaction...,"{'Author': '', 'CreationDate': 'D:202012230158..."


In [77]:
df_cleaned = df_parsed.copy()

In [78]:
# ensure the colums are string types
df_cleaned["pdf_text"] = df_cleaned["pdf_text"].astype(str)

In [79]:
df_cleaned['pdf_text'] = (
    df_cleaned['pdf_text']
      .str.replace(r"\n", " ", regex=False)
      .str.strip()
)

In [80]:
df_cleaned.head()

,pdf_text,metadata
0,['NATURAL CALCIUM CARBONATE FOR BIOMEDICAL APP...,"{'Author': 'ismail - [2010]', 'Creator': 'Micr..."
1,['ProceedingsofMachineLearningResearchLEAVEUNS...,"{'Author': '', 'CreationDate': 'D:202404290022..."
2,['Preprint Advances in Chemical Physics Vol. 1...,"{'CreationDate': 'D:20060424095709Z', 'ModDate..."
3,['Enhancing Drug-Drug Interaction Extraction f...,"{'Author': '', 'CreationDate': 'D:201805160029..."
4,['BERTChem-DDI : Improved Drug-Drug Interactio...,"{'Author': '', 'CreationDate': 'D:202012230158..."


In [83]:
df_cleaned.tail()

,pdf_text,metadata
94,['Predicting Drug-Drug Interactions using Deep...,"{'Author': '', 'CreationDate': 'D:202211010140..."
95,['An Edge-based Graph Transformer Network for ...,"{'Author': 'SuperMan', 'Company': '', 'Creatio..."
96,['ChatGPT-powered Conversational Drug Editing ...,"{'Author': '', 'CreationDate': 'D:202305300039..."
97,['HeTriNet: Heterogeneous Graph Triplet Attent...,"{'Author': '', 'CreationDate': 'D:202312040149..."
98,['DRUG-TARGET INTERACTION/AFFINITY PREDICTION:...,"{'Author': 'Ali Vefghi, Zahed Rahmati, Mohamma..."


In [81]:
df_cleaned.to_csv("extracted_pdf_info.csv", index=False)

In [85]:
from bertopic import BERTopic

topics, _ = BERTopic().fit_transform(df_cleaned["pdf_text"])
df_cleaned["topic"] = topics